In [1]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd



In [2]:
test = np.loadtxt("mnist_test.csv", delimiter = ",")
train = np.loadtxt("mnist_train.csv", delimiter = ",")

In [3]:
train_X, train_Y = train[:,1:].reshape(60000, 28 * 28 ), train[:,0]
test_X, test_Y = test[:,1:].reshape(10000, 28 * 28 ), test[:,0]

train_X.shape, train_Y.shape, test_X.shape, test_Y.shape



((60000, 784), (60000,), (10000, 784), (10000,))

In [4]:
model = svm.SVC(C=20)
model.fit(train_X, train_Y)

SVC(C=20, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [5]:
preds = model.predict(test_X)

In [6]:
accuracy_score(preds, test_Y)

0.9834

In [50]:
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor


In [62]:
transform = transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])

trainset = datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True)



In [63]:
testset = datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False)

In [127]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 128, 3, 1)
        self.conv3 = nn.Conv2d(128, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(5 * 5 * 64, 128)
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output
    
    

In [128]:
net = Net()

In [129]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=1e-3, momentum=0.9)


In [130]:
running_loss = 0
for epochs in range(2):
    for i, data in enumerate(trainloader):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 0:  
            print('[%d %5d] loss: %.3f' %
                  (epochs, i + 1, running_loss / 100))
            running_loss = 0.0

[0     1] loss: 0.023
[0   101] loss: 2.271
[0   201] loss: 1.910
[0   301] loss: 1.062
[0   401] loss: 0.738
[0   501] loss: 0.602
[0   601] loss: 0.494
[0   701] loss: 0.422
[0   801] loss: 0.383
[0   901] loss: 0.382
[0  1001] loss: 0.334
[0  1101] loss: 0.322
[0  1201] loss: 0.287
[0  1301] loss: 0.279
[0  1401] loss: 0.290
[0  1501] loss: 0.252
[0  1601] loss: 0.246
[0  1701] loss: 0.228
[0  1801] loss: 0.200
[1     1] loss: 0.173
[1   101] loss: 0.189
[1   201] loss: 0.220
[1   301] loss: 0.193
[1   401] loss: 0.208
[1   501] loss: 0.166
[1   601] loss: 0.189
[1   701] loss: 0.189
[1   801] loss: 0.181
[1   901] loss: 0.178
[1  1001] loss: 0.177
[1  1101] loss: 0.190
[1  1201] loss: 0.167
[1  1301] loss: 0.170
[1  1401] loss: 0.144
[1  1501] loss: 0.153
[1  1601] loss: 0.152
[1  1701] loss: 0.136
[1  1801] loss: 0.152


In [131]:
num_true= 0
with torch.no_grad():
    for data in testloader:
        images,labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        num_true += len(np.argwhere((predicted == labels).numpy() == True))
        print(len(np.argwhere((predicted == labels).numpy() == True)))
        


31
32
31
32
30
31
31
31
30
31
29
29
31
29
30
31
31
31
29
30
31
29
31
31
31
31
30
28
30
26
30
31
31
30
28
32
30
31
28
31
28
29
32
31
31
32
31
28
29
31
29
31
29
29
29
30
30
31
31
28
31
29
32
27
32
29
28
31
28
31
30
30
29
32
26
30
31
31
31
31
32
29
30
32
31
31
30
31
31
32
31
31
30
29
32
30
31
31
31
32
30
31
31
32
26
30
30
32
31
30
28
28
30
30
32
32
30
30
27
29
29
31
30
28
29
30
31
30
30
32
30
30
28
30
29
31
28
32
31
32
28
30
30
32
30
32
30
30
30
31
29
29
28
32
31
30
32
32
30
32
32
32
32
30
31
31
31
32
32
32
31
32
32
32
31
31
30
31
32
30
32
31
30
31
32
31
28
30
29
30
31
31
29
32
31
32
32
32
32
32
32
32
32
30
30
29
28
29
31
31
31
30
31
29
32
31
30
32
32
31
32
31
32
32
31
31
31
31
31
32
32
32
29
32
32
32
32
31
32
32
32
31
32
30
30
30
29
29
31
29
31
32
30
32
32
30
32
32
30
30
29
32
29
32
31
32
31
32
32
32
32
32
32
32
32
32
31
32
32
32
32
29
28
31
30
32
32
32
31
32
32
32
32
31
30
31
32
31
30
32
31
29
30
29
30
27
30
31
32
29
32
31
16


In [134]:
print("accuracy: ",num_true / 10000)

accuracy:  0.9565
